## Connect with Google Drive

In [ ]:
!pip install gdown

In [ ]:
import gdown

url = "https://drive.google.com/uc?id=1RhakmYA0l99TEQLbHan9rfEyj12hPYL8"

output = 'video.mp4'

gdown.download(url, output)

In [ ]:
SOURCE_VIDEO_PATH = "/kaggle/working/video.mp4"

## Check GPU access

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install YOLOv8

In [ ]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

## Install Roboflow Supervision

In [ ]:
!pip install supervision==0.3.0

from IPython import display
display.clear_output()

import supervision
print("supervision.__version__:", supervision.__version__)

## Load pre-trained YOLOv8 model

In [ ]:
%cd {HOME}

In [ ]:
import gdown

url = "https://drive.google.com/uc?id=11CVHlga1WsmH3MsytW-ItWiCGwYZ3G98"

output = '/kaggle/working/trained_model.pt'

gdown.download(url, output)

In [ ]:
from ultralytics import YOLO

model = YOLO(os.path.relpath("/kaggle/working/trained_model.pt"))
model.fuse()

## Track and Count

In [ ]:
import supervision as sv

LINE_START = sv.Point(0, 900)
LINE_END = sv.Point(1000, 480)

In [ ]:
TARGET_VIDEO_PATH = f"/kaggle/working/output_video.mp4"

In [ ]:
# create LineCounter instance
line_counter = sv.LineZone(start=LINE_START, end=LINE_END)

# create instance of BoxAnnotator and LineCounterAnnotator
line_annotator = sv.LineZoneAnnotator(thickness=2, text_thickness=1, text_scale=0.5)
box_annotator = sv.BoxAnnotator(
    thickness=2,
    text_thickness=1,
    text_scale=0.5
)

# create VideoInfo instance
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# create frame generator
generator = sv.video.get_video_frames_generator(SOURCE_VIDEO_PATH)

# open target video file
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:  
    for result in model.track(source=SOURCE_VIDEO_PATH, show=False, stream=True, agnostic_nms=True):

        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        if result.boxes.id is not None:
            detections.tracker_id = result.boxes.id.cpu().numpy().astype(int)

        detections = detections[(detections.class_id != 60) & (detections.class_id != 0)]

        labels = [
            f"{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
            for _, confidence, class_id, tracker_id
            in detections
        ]

        frame = box_annotator.annotate(
            scene=frame, 
            detections=detections,
            labels=labels
        )

        line_counter.trigger(detections=detections)
        line_annotator.annotate(frame=frame, line_counter=line_counter)
        sink.write_frame(frame)

In [ ]:
from IPython.display import FileLink
FileLink(r'output_video.mp4')